# CUNEF MUCD 2021/2022  
## Machine Learning
## Análisis de Siniestralidad de Automóviles

### Autores:
- Andrés Mahía Morado
- Antonio Tello Gómez


# Modelo Base y Naive Bayes

En este apartado construimos el modelo base que nos servirá como referencia para comprobar el desempeño del resto de modelos. Por otro lado, también vamos entrenar un modelo Naive Bayes otro modelo sencillo que también nos servirá como referencia. 

In [8]:
#Librerías
import pandas as pd
import numpy as np

from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, \
                            silhouette_score, recall_score, precision_score, make_scorer, \
                            roc_auc_score, f1_score, precision_recall_curve
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, roc_auc_score, \
                            classification_report, confusion_matrix
from sklearn import metrics
from sklearn.metrics import plot_confusion_matrix

from sklearn.dummy import DummyClassifier 
from sklearn.naive_bayes import GaussianNB

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

import pickle
import warnings
warnings.filterwarnings('ignore')
%load_ext autotime

from aux_func import evaluate_model
from aux_func import cargar_modelo

time: 0 ns (started: 2021-12-18 20:07:20 +01:00)


In [9]:
xtrain = pd.read_parquet("../data/xtrain.parquet")
ytrain = pd.read_parquet("../data/ytrain.parquet")
xtest = pd.read_parquet("../data/xtest.parquet")
ytest = pd.read_parquet("../data/ytest.parquet")

time: 1.67 s (started: 2021-12-18 20:07:22 +01:00)


In [10]:
#Cargamos pipeline preprocesado
preprocessor = cargar_modelo('../models/preprocessor.pickle')

time: 16 ms (started: 2021-12-18 20:07:24 +01:00)


# Modelo Base

Un modelo base emplea un valor fijo o valores aleatorios (Normalmente en función de algún estadístico de la distribución de la variable objetivo) para realizar la predicción de los valores del test. 
En nuestro caso hemos empleado la opción 'stratified' que genera predicciones aleatorias respetando la distribución de la variable objetivo.  


In [12]:
modelo_base = Pipeline(steps=[
    ('preprocesador', preprocessor),
    ('clasificador', DummyClassifier(strategy='stratified', random_state=0))])

time: 0 ns (started: 2021-12-18 20:08:05 +01:00)


In [13]:
modelo_base.fit(xtrain, ytrain)

Pipeline(steps=[('preprocesador',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['vehicle_age',
                                                   'passenger_age',
                                                   'vehicles_involved',
                                                   'year']),
                                                 ('fcat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=nan,
                                                                

time: 25.8 s (started: 2021-12-18 20:08:07 +01:00)


In [14]:
with open('../models/modelo_base.pickle', 'wb') as f:
    pickle.dump(modelo_base, f)

time: 0 ns (started: 2021-12-18 20:08:33 +01:00)


In [15]:
cargar_modelo('../models/modelo_base.pickle')

Pipeline(steps=[('preprocesador',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['vehicle_age',
                                                   'passenger_age',
                                                   'vehicles_involved',
                                                   'year']),
                                                 ('fcat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=nan,
                                                                

time: 62 ms (started: 2021-12-18 20:08:33 +01:00)


In [16]:
ypred = modelo_base.predict(xtest)
ypred_proba = modelo_base.predict_proba(xtest)
evaluate_model(ytest,ypred,ypred_proba)

ROC-AUC score of the model: 0.5006305803052006
Accuracy of the model: 0.9702512936495137

Classification report: 
              precision    recall  f1-score   support

           0       0.98      0.98      0.98    799946
           1       0.02      0.02      0.02     12291

    accuracy                           0.97    812237
   macro avg       0.50      0.50      0.50    812237
weighted avg       0.97      0.97      0.97    812237


Confusion matrix: 
[[787873  12073]
 [ 12090    201]]

time: 7.19 s (started: 2021-12-18 20:08:33 +01:00)


Algo muy significativo de nuestro modelo base es que obtenemos un accuracy, la métrica por excelencia para evaluar los modelos de ML, del 97.02%. Si predijéramos que todo nuestro test pertenece a la clase 0 (strategy='most_frequent') obtendríamos un accuracy aún mayor, del 98.47%. Sin embargo, no estaríamos acercándonos de ninguna manera a nuestro objetivo que es predecir el máximo posible de la clase 1 sin renunciar a acertar en la clase 0. Por tanto, el accuracy no será una métrica muy útil a la hora de evaluar nuestros modelos.

# Naive Bayes

El Naive Bayes es un clasificador basado en la aplicación del teorema de Bayes con la hipótesis de independencia condicional entre variables predictoras. Hipótesis conocida como ingenua, _naive_.  
En nuestro caso hemos estimado un Gaussian Naive Bayes en el que asumimos que probabilidad de las variables predictoras sigue una distribución normal.  
Este es el modelo más sencillo que tiene en cuenta las variables predictoras por ello también será una referencia muy útil para comparar el resto de modelos.

In [17]:
naive_bayes = Pipeline(steps=[
    ('preprocesador', preprocessor),
    ('clasificador', GaussianNB())])

time: 0 ns (started: 2021-12-18 20:09:26 +01:00)


In [18]:
naive_bayes.fit(xtrain, ytrain)

Pipeline(steps=[('preprocesador',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['vehicle_age',
                                                   'passenger_age',
                                                   'vehicles_involved',
                                                   'year']),
                                                 ('fcat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=nan,
                                                                

time: 27.9 s (started: 2021-12-18 20:09:35 +01:00)


In [19]:
with open('../models/GNB.pickle', 'wb') as f:
    pickle.dump(naive_bayes, f)

time: 0 ns (started: 2021-12-18 20:10:03 +01:00)


In [20]:
# Para no tener que ejecutar, saltarse el fit y ejecutar a partir de aquí
cargar_modelo('../models/GNB.pickle')

Pipeline(steps=[('preprocesador',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['vehicle_age',
                                                   'passenger_age',
                                                   'vehicles_involved',
                                                   'year']),
                                                 ('fcat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=nan,
                                                                

time: 62 ms (started: 2021-12-18 20:10:03 +01:00)


In [22]:
ypred = naive_bayes.predict(xtest)
ypred_proba = naive_bayes.predict_proba(xtest)
evaluate_model(ytest,ypred,ypred_proba)

ROC-AUC score of the model: 0.6915075906932969
Accuracy of the model: 0.8819839529595426

Classification report: 
              precision    recall  f1-score   support

           0       0.99      0.89      0.94    799946
           1       0.03      0.24      0.06     12291

    accuracy                           0.88    812237
   macro avg       0.51      0.56      0.50    812237
weighted avg       0.97      0.88      0.92    812237


Confusion matrix: 
[[713478  86468]
 [  9389   2902]]

time: 8.25 s (started: 2021-12-18 20:10:23 +01:00)


## Ajuste del umbral de predicción

In [23]:
#NEW
# keep probabilities for the positive outcome only
yhat = ypred_proba[:, 1]
# calculate roc curves
fpr, tpr, thresholds = roc_curve(ytest, yhat)

gmeans = np.sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = np.argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))

ypred_new_threshold = (ypred_proba[:,1]>thresholds[ix]).astype(int)
evaluate_model(ytest,ypred_new_threshold,ypred_proba)

Best Threshold=0.001171, G-Mean=0.659
ROC-AUC score of the model: 0.6915075906932969
Accuracy of the model: 0.6182862883616481

Classification report: 
              precision    recall  f1-score   support

           0       0.99      0.62      0.76    799946
           1       0.03      0.70      0.05     12291

    accuracy                           0.62    812237
   macro avg       0.51      0.66      0.41    812237
weighted avg       0.98      0.62      0.75    812237


Confusion matrix: 
[[493556 306390]
 [  3652   8639]]

time: 1.59 s (started: 2021-12-18 20:10:38 +01:00)
